In [ ]:
import os
results_dir='RESULTS_WEEK_1'
if not os.path.exists(results_dir):
    os.mkdir(results_dir)


In [ ]:
# DAY_2 Fetch ohhlcv data and check the MA and calculate rolling volaitlity and plot them 

# Import necessary libraries
import ccxt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

binance = ccxt.binance()

# Fetch the OHLCV data for a given symbol
symbols=['BTC/USDT','ETH/USDT','BNB/USDT']

ohlcv_data = {}

for symbol in symbols:
    try:
        ohlcv = binance.fetch_ohlcv(symbol, timeframe='1h', limit=1000)
        ohlcv=pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        ohlcv['timestamp'] = pd.to_datetime(ohlcv['timestamp'], unit='ms')
        ohlcv_data[symbol] = ohlcv
    except Exception as e:
        print(f'Error fetching OHLCV data for {symbol}: {str(e)}')
DATA=pd.concat(ohlcv_data,axis=1)
DATA.columns = ['_'.join(col).strip() for col in DATA.columns.values]
DATA['DATE']=DATA['BTC/USDT_timestamp']
DATA.set_index('DATE',inplace=True)
columns_to_drop = ['BTC/USDT_timestamp', 'ETH/USDT_timestamp', 'BNB/USDT_timestamp']
DATA.drop(columns=columns_to_drop, inplace=True)


# calculating log returns moving average and rolling volatility
for symbol in symbols:
    DATA[f'{symbol}_log_return'] = np.log(DATA[f'{symbol}_close'] / DATA[f'{symbol}_close'].shift(1))
    DATA[f'{symbol}_MA_20'] = DATA[f'{symbol}_close'].rolling(window=20).mean()
    DATA[f'{symbol}_volatility_20'] = DATA[f'{symbol}_log_return'].rolling(window=20).std()*np.sqrt(252)
    DATA[f'{symbol}_MA_50'] = DATA[f'{symbol}_close'].rolling(window=50).mean()
    

# Plotting the mv=oving avergaes and volaltitly

plt.style.use('ggplot')
fig, ax = plt.subplots(len(symbols),1, figsize=(15, 10))

file_path=os.path.join(results_dir, 'moving_averages.png')
for i, symbol in enumerate(symbols):
    ax[i].plot(DATA[f'{symbol}_close'], label=f'{symbol} close')
    ax[i].plot(DATA[f'{symbol}_MA_20'], label=f'{symbol} MA_20')
    ax[i].plot(DATA[f'{symbol}_MA_50'], label=f'{symbol} MA_50')
    ax[i].set_title(f'{symbol} price and moving averages')
    ax[i].legend()
    ax[i].grid(True)
    fig.subplots_adjust(hspace=0.5)
    
fig.subplots_adjust(hspace=0.5)
plt.savefig(file_path)  

In [ ]:

                                            # DAY 3 PORTFOLIO RISK AND RETURN ANALYSIS

# IMPORT USEFUL LIBRARIES

import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
import ccxt
                                                # FETCHING OHLCV DATA FOR GIVEEN COINS
binance=ccxt.binance()
symbols=['BTC/USDT','ETH/USDT','SOL/USDT']
ohlcv_data={}
for symbol in symbols:
    try:
        ohlcv=binance.fetch_ohlcv(symbol,timeframe='1d',limit=1000)
        ohlcv=pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        ohlcv['timestamp'] = pd.to_datetime(ohlcv['timestamp'], unit='ms')
        ohlcv_data[symbol]=ohlcv
    except Exception as e:
        print(f'Error fetching OHLCV data for {symbol}: {str(e)}')

DATA=pd.concat(ohlcv_data,axis=1)
DATA.columns = ['_'.join(col).strip() for col in DATA.columns.values]
DATA['DATE']=DATA['BTC/USDT_timestamp']
DATA.set_index('DATE',inplace=True)
columns_to_drop = ['BTC/USDT_timestamp', 'ETH/USDT_timestamp', 'SOL/USDT_timestamp']
DATA.drop(columns=columns_to_drop, inplace=True)
PORTFOLIO_DATA={}
RETURN_DATA=pd.DataFrame()
SHARPE_RATIOS={}

for symbol in symbols:
    DATA[f'{symbol}_log_return'] = np.log(DATA[f'{symbol}_close'] / DATA[f'{symbol}_close'].shift(1))
    PORTFOLIO_DATA[symbol]={}
    PORTFOLIO_DATA[symbol]['LOG_RETURN'] = DATA[f'{symbol}_log_return'].mean()*252
    PORTFOLIO_DATA[symbol]['VOLATILITY'] =DATA[f'{symbol}_log_return'].std()*np.sqrt(252)
    PORTFOLIO_DATA[symbol]['WEIGHT'] = 1/len(symbols)
    RETURN_DATA[symbol]=DATA[f'{symbol}_log_return']
    SHARPE_RATIOS[symbol]=PORTFOLIO_DATA[symbol]['LOG_RETURN']/PORTFOLIO_DATA[symbol]['VOLATILITY']
   
PORTFOLIO_DATA=pd.DataFrame(PORTFOLIO_DATA).T 
PORTFOLIO_RESULTS={}
    
PORTFOLIO_RETURN=(PORTFOLIO_DATA['LOG_RETURN']*PORTFOLIO_DATA['WEIGHT']).sum()


COV_MATRIX_ANNUAL=RETURN_DATA.cov()*252
WEIGHTS=np.array([1/len(symbols)]*len(symbols))




                                    # PORTFOLIO RESULTS LIKE SHARPE RATIO RETURNS AND VOLALITILITY

PORTFOLIO_RESULTS['RETURNS']=(PORTFOLIO_DATA['LOG_RETURN']*PORTFOLIO_DATA['WEIGHT']).sum()
PORTFOLIO_RESULTS['VOLATILITY']=np.sqrt(WEIGHTS.T @ COV_MATRIX_ANNUAL @ WEIGHTS)
PORTFOLIO_RESULTS['SHARPE_RATIO']=PORTFOLIO_RETURN/PORTFOLIO_RESULTS['VOLATILITY']
PORTFOLIO_RESULTS.items()

                                    # Plotting the Sharpe Ratios   

fig, ax = plt.subplots(1, 1, figsize=(10, 6))
plt.style.use('dark_background')
file_path=os.path.join(results_dir, 'sharpe_ratios_Equalweighted_portfolio_sharpe_ratios.png')                                                   

ax.bar(SHARPE_RATIOS.keys(), SHARPE_RATIOS.values(), color='#f8f4e3',width=0.5,label='Sharpe Ratios',edgecolor='black',linewidth=1.2)
plt.bar('PORTOLIO_SHARPE_RATIO',PORTFOLIO_RESULTS['SHARPE_RATIO'],color='#d4af37',width=0.5,label='PORTOLIO_SHARPE_RATIO',edgecolor='black',linewidth=1.2)

ax.set_title('Sharpe Ratios')
ax.set_xlabel('Symbols')
ax.set_ylabel('Sharpe Ratio')
ax.legend(loc='upper right',frameon=True)
plt.tight_layout
ax.grid(True,alpha=0.5,color='r')
plt.savefig(file_path, dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# DAY 4 PORTFOLIO OPTIMIZATION , EFFICIENT FRONTIER ,TANGENCY PORTFOLIO AND PLOT THEM
                                        
                                        
                                        # Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ccxt
import scipy.optimize as sco
import scipy.stats as stats
                                    # FETCH DATA AND CALCULAYE THE LOG RETURN AND VOLATILITY
binance = ccxt.binance()
symbols = ['ETH/USDT', 'SOL/USDT','BNB/USDT','XRP/USDT']
CLOSE_DATA = pd.DataFrame()
PORTFOLIO_DATA={}
WEIGHTS=np.array([1/len(symbols)]*len(symbols))

for symbol in symbols:
    try:
        ohlcv = binance.fetch_ohlcv(symbol, timeframe='1d', limit=1000)
        ohlcv = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        ohlcv['DATE'] = pd.to_datetime(ohlcv['timestamp'], unit='ms')
        CLOSE_DATA[symbol] = ohlcv['close']

       
        
    except Exception as e:
        print(f'Error fetching OHLCV data for {symbol}: {str(e)}')

CLOSE_DATA.dropna(inplace=True)
CLOSE_DATA.set_index(ohlcv['DATE'], inplace=True)
CLOSE_DATA
LOG_RETURN_DATA = pd.DataFrame()
MEAN_LOG_RETURN = {}
                                
                                
                                # CALCUALTIN THE PORTFOLIO RETUN AND VOLATILITY 
for symbol in symbols:
    LOG_RETURN_DATA[f'{symbol}'] = np.log(CLOSE_DATA[symbol] / CLOSE_DATA[symbol].shift(1))
    PORTFOLIO_DATA[symbol] = {}
    PORTFOLIO_DATA[symbol]['LOG_RETURN'] =  LOG_RETURN_DATA[f'{symbol}'] .mean() * 252
    PORTFOLIO_DATA[symbol]['VOLATILITY'] = LOG_RETURN_DATA[f'{symbol}'].std() * np.sqrt(252)
    PORTFOLIO_DATA[symbol]['WEIGHT'] = 1 / len(symbols)
    MEAN_LOG_RETURN[symbol] = PORTFOLIO_DATA[symbol]['LOG_RETURN'] 
    
# CALCULATING THE COPVARIENCE MATRUIX AND CORRELATION MATRIX
COV_MATRIX_ANNUAL = LOG_RETURN_DATA.cov() * 252
CORRELATION_MATRIX = LOG_RETURN_DATA.corr()
COV_MATRIX_ANNUAL

# INITIAL RETURN AND VOLATILITY VALUE 
INITIAL_VOLAITLITY= np.sqrt(WEIGHTS.T @ COV_MATRIX_ANNUAL @ WEIGHTS)
MEAN_LOG_RETURN=pd.DataFrame(MEAN_LOG_RETURN.values(),index=MEAN_LOG_RETURN.keys(),columns=['LOG_RETURN'])
INITIAL_RETURN=MEAN_LOG_RETURN.values.T @ WEIGHTS
INITIAL_sharpe_ratio = INITIAL_RETURN / INITIAL_VOLAITLITY


# minimum varince portfolio using the scipy.optimise library

constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = tuple((0, 1) for x in range(len(symbols)))

optimal_weights = sco.minimize(
    lambda x: np.sqrt(x.T @ COV_MATRIX_ANNUAL @ x),
    WEIGHTS,
    method='SLSQP',
    bounds=bounds,
    constraints=constraints
)


                                # Calcuating the optimal weights, return, volatility and sharpe ratio

optimal_weights=optimal_weights.x
optimial_return= MEAN_LOG_RETURN.values.T @ optimal_weights
optimial_volatility=np.sqrt(optimal_weights.T @ COV_MATRIX_ANNUAL @ optimal_weights)
optimal_sharpe_ratio=optimial_return/optimial_volatility


                                    # GENERATING THE EFFCIENT FRONTIER AND RANDOM PORTFOLIO WEIGHTS 
GENERATORS=1000
GEN_WEIGHTS=np.random.standard_normal(size=(GENERATORS,len(symbols)))
GEN_WEIGHTS=np.abs(GEN_WEIGHTS)
for i in range(GEN_WEIGHTS.shape[0]):
    GEN_WEIGHTS[i]=GEN_WEIGHTS[i]/np.sum(GEN_WEIGHTS[i])

RETURNS=GEN_WEIGHTS @ MEAN_LOG_RETURN.values

VOLATILITIES=np.sqrt(np.diag(GEN_WEIGHTS @ COV_MATRIX_ANNUAL.values @ GEN_WEIGHTS.T))


                                            # GENERATING THE TANGENCY PORTFOLIO
risk_free_rate = 0.05
def negative_sharpe_ratio(weights,  MEAN_LOG_RETURN, cov_matrix, risk_free_rate):
    portfolio_return = weights @ MEAN_LOG_RETURN.values
    portfolio_volatility = np.sqrt(weights.T @ cov_matrix @ weights)
    return -(portfolio_return - risk_free_rate) / portfolio_volatility
    


                                            # CACULATING THE TANGENCY PORTFOLIO WEIGHTS 
bounds = tuple((0, 1) for x in range(len(symbols)))
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

WEIGHTS_tangency=sco.minimize(negative_sharpe_ratio, WEIGHTS, args=(MEAN_LOG_RETURN, COV_MATRIX_ANNUAL, risk_free_rate), method='SLSQP', bounds=bounds, constraints=constraints).x
TANGENCY_PORTFOLIO_RETURN = MEAN_LOG_RETURN.values.T @ WEIGHTS_tangency
TANGENCY_PORTFOLIO_VOLATILITY = np.sqrt(WEIGHTS_tangency.T @ COV_MATRIX_ANNUAL @ WEIGHTS_tangency)




                                            # GENERARTIN THE CAPITAL ALOCATIION LINE THE TANGENCY LINE

SIGMA_RANGE=np.linspace(0.4, 0.8, 100)
TANGENCY_LINE = risk_free_rate + (TANGENCY_PORTFOLIO_RETURN - risk_free_rate) / TANGENCY_PORTFOLIO_VOLATILITY * SIGMA_RANGE





# GENERATING THE EFFICIENT FRONTIER
TARGET_RETURNS = np.linspace(RETURNS.min(), RETURNS.max(), 100)
def portfolio_volatility(weights, mean_returns, cov_matrix):
    return np.sqrt(weights.T @ cov_matrix @ weights)

def portfolio_return(weights, mean_returns):
    return weights @ mean_returns.values

def optimal_volatility(MEAN_LOG_RETURN,TARGET_RETURNS, cov_matrix,weights=WEIGHTS):
    bounds = tuple((0, 1) for x in range(len(symbols)))
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
                   {'type': 'eq', 'fun': lambda x: portfolio_return(x, MEAN_LOG_RETURN) - TARGET_RETURNS})

                               
                               
    result = sco.minimize(portfolio_volatility, weights, args=(MEAN_LOG_RETURN, cov_matrix), method='SLSQP', bounds=bounds, constraints=constraints)
    return result.fun                           
                               
TARGET_VOLATILITY = np.array([optimal_volatility(MEAN_LOG_RETURN, target_return, COV_MATRIX_ANNUAL) for target_return in TARGET_RETURNS])                
                               
                                
                                
                                
                                
                                # PLOTING THE EFFICIENT FRONTIER AND TANGENCY LINE aND RANDOM PORTFOLIOS and tangency portfolio 
plt.style.use('dark_background')
fig, ax = plt.subplots(figsize=(12, 8))
file_path=os.path.join(results_dir, 'efficient_frontier_Tangency_portfolio.png')

ax.scatter(VOLATILITIES, RETURNS, cmap='viridis', marker='o')
ax.scatter(INITIAL_VOLAITLITY, INITIAL_RETURN, color='red', marker='o', label='Initial Portfolio',s=100,linewidths=0.5,edgecolors='white')
ax.scatter(optimial_volatility, optimial_return, color='blue', marker='o', label='Optimal Portfolio',s=100,linewidths=0.5,edgecolors='white')
ax.plot(TARGET_VOLATILITY, TARGET_RETURNS, color='orange', label='Efficient Frontier',linewidth=2.5, alpha=0.8)
ax.plot(SIGMA_RANGE, TANGENCY_LINE, color='green', label='Capital Allocation Line',linewidth=2.5, alpha=0.8)
ax.scatter(TANGENCY_PORTFOLIO_VOLATILITY, TANGENCY_PORTFOLIO_RETURN, color='purple', marker='o', label='Tangency Portfolio',s=100,linewidths=0.5,edgecolors='white')
ax.set_xlabel('Log Volatility')
ax.set_ylabel('Expected Log Return')
ax.legend(loc='upper left', frameon=True)
ax.annotate('Initial Portfolio', xy=(INITIAL_VOLAITLITY, INITIAL_RETURN), xytext=(INITIAL_VOLAITLITY+0.01, INITIAL_RETURN+0.01), arrowprops=dict(facecolor='brown', shrink=0.05))
ax.annotate('Optimal Portfolio', xy=(optimial_volatility, optimial_return), xytext=(optimial_volatility+0.01, optimial_return+0.01), arrowprops=dict(facecolor='brown', shrink=0.05))
ax.annotate('Tangency Portfolio', xy=(TANGENCY_PORTFOLIO_VOLATILITY, TANGENCY_PORTFOLIO_RETURN), xytext=(TANGENCY_PORTFOLIO_VOLATILITY+0.01, TANGENCY_PORTFOLIO_RETURN+0.01), arrowprops=dict(facecolor='brown', shrink=0.05))
ax.set_title('Efficient Frontier and Random Portfolios')    
ax.grid(True, alpha=0.5, color='r')
plt.savefig(file_path, dpi=300, bbox_inches='tight')
plt.tight_layout()
plt.show()

In [ ]:
# DAY 5 MONTE CARLO SIMULATIION VS BLACK SCHOLES PRICING MODEL COMPARISON
                                    # Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

                                  # DEfining important quantitites like volaltutlity risk free rate innitial stok price an strike price 
s0 = 100                          # Initial stock price
strikes = np.arange(80, 200, 5)   # Strike prices from 80 to 200
risk_free_rate = 0.05             # Risk-free interest rate
volatility= 0.5                   # Volatility of the underlying asset
Duration = 1                      # Time to expiration in years


                                    # defingin the black scholes pricing model function
def BSM(type,stock_price, strike_price, risk_free_rate, volatility, duration):
    type=type.lower()
    if type not in ['call', 'put']:
        raise ValueError("Option type must be either 'call' or 'put'")
    d1 = (np.log(stock_price / strike_price) + (risk_free_rate + 0.5 * volatility ** 2) * duration) / (volatility * np.sqrt(duration))
    d2 = d1 - volatility * np.sqrt(duration)
    if type == 'call':
        price = (stock_price * stats.norm.cdf(d1) - strike_price * np.exp(-risk_free_rate * duration) * stats.norm.cdf(d2))
    elif type == 'put':
        price = (strike_price * np.exp(-risk_free_rate * duration) * stats.norm.cdf(-d2) - stock_price * stats.norm.cdf(-d1))
    return price

                                # DEFininht the path gneartion fucnction 
def path_generations(Noofpaths,Noofsteps,stock_price, strike_price, risk_free_rate, volatility, duration):
    dt = duration / Noofsteps
    S= np.zeros((Noofpaths, Noofsteps + 1))
    z=np.random.normal(size=(Noofpaths, Noofsteps))
    S[:, 0] = stock_price
    time=[i*dt for i in range(Noofsteps+1)]
    results={}
    for i in range(Noofsteps):
        z[:,i]=(z[:,i]-np.mean(z[:,i]))/np.std(z[:,i])
        S[:,i+1]=S[:,i]+( risk_free_rate*S[:,i]*dt + volatility*S[:,i]*np.sqrt(dt)*z[:,i])
        
    results['paths']=S
    results['time']=time
    results['stock_T']=S[:,-1]
    return results
                                # Definin hte monte carlo pricing fucntion
def MONTE_CARLO_PRICING(type,Noofpaths,Noofsteps,stock_price, strike_price, risk_free_rate, volatility, duration):
    results=path_generations(Noofpaths,Noofsteps,stock_price, strike_price, risk_free_rate, volatility, duration)
    stock_T=results['stock_T']
    type=type.lower()
    if type not in ['call', 'put']:
        raise ValueError("Option type must be either 'call' or 'put'")
    if type == 'call':
        payoff = np.mean(np.maximum(stock_T - strike_price, 0))
    elif type == 'put':
        payoff = np.mean(np.maximum(strike_price - stock_T, 0))
    price = np.exp(-risk_free_rate * duration) * payoff
    return price

# CALCUALTING THE CALL AND PUT PRICES FOR DIFFERENT STRIKES USING THE BLACK SCHOLES MODEL AND MONTE CARLO SIMULATION
PRICES=pd.DataFrame(index=strikes)

PRICES['Black Scholes Call Price'] = [BSM('call', s0, strike, risk_free_rate, volatility, Duration) for strike in strikes]
PRICES['Black Scholes Put Price'] = [BSM('put', s0, strike, risk_free_rate, volatility, Duration) for strike in strikes]
PRICES['Monte Carlo Call Price'] = [MONTE_CARLO_PRICING('call', 10000, 100, s0, strike, risk_free_rate, volatility, Duration) for strike in strikes]
PRICES['Monte Carlo Put Price'] = [MONTE_CARLO_PRICING('put', 10000, 100, s0, strike, risk_free_rate, volatility, Duration) for strike in strikes]
PRICES['PERCENT_ERROR_CALL_PRICES']= (PRICES['Black Scholes Call Price'] - PRICES['Monte Carlo Call Price'])/PRICES['Black Scholes Call Price']*100
PRICES['PERCENT_ERROR_PUT_PRICES']= (PRICES['Black Scholes Put Price'] - PRICES['Monte Carlo Put Price'])/PRICES['Black Scholes Put Price']*100
PRICES.index.name = 'Strike Price'


file_path=os.path.join(results_dir, 'BLACK_SCHOLES_MONTE_CARLO_PRICES_CALL_PUT.csv')
PRICES.to_csv(file_path)


# PLOTTING THE CALL AND PUT PRICES FOR DIFFERENT STRIKES USING THE BLACK SCHOLES MODEL AND MONTE CARLO SIMULATION
plt.style.use('dark_background')
fig, ax = plt.subplots(figsize=(12, 8))
file_path=os.path.join(results_dir, 'black_scholes_vs_monte_carlo.png')
ax.plot(PRICES.index, PRICES['Black Scholes Call Price'], label='Black Scholes Call Price', color='blue')
ax.plot(PRICES.index, PRICES['Monte Carlo Call Price'], label='Monte Carlo Call Price', color='red')
ax.plot(PRICES.index, PRICES['Black Scholes Put Price'], label='Black Scholes Put Price', color='green')
ax.plot(PRICES.index, PRICES['Monte Carlo Put Price'], label='Monte Carlo Put Price', color='orange')
ax.set_xlabel('Strike Price',fontdict={'fontsize': 14, 'fontweight': 'bold'})
ax.set_ylabel('Option Price',fontdict={'fontsize': 14, 'fontweight': 'bold'})
ax.set_title('Black Scholes vs Monte Carlo Pricing',fontdict={'fontsize': 14, 'fontweight': 'bold'})
ax.legend()
ax.grid(True, alpha=0.5, color='r')
plt.savefig(file_path, dpi=300, bbox_inches='tight')
plt.tight_layout()



# STOCK PATH SIMULATIONS TO SHOW THE STOCHSATIC NTURE OF THE STOCK PRICE AND SLO THE NATURE OF THE DSITRUBTUIIO STCOCK PROCE FOLLOWS

plt.style.use('dark_background')
fig, ax = plt.subplots(figsize=(12, 8))
file_path=os.path.join(results_dir, 'stock_path_simulations.png')
paths=path_generations(1000,100,s0,strikes[0],risk_free_rate,volatility,Duration)['paths']
time=path_generations(1000,100,s0,strikes[0],risk_free_rate,volatility,Duration)['time']
for i in range(100):
    ax.plot(time, paths[i,:], color='royalblue', alpha=0.8)
ax.grid(True, alpha=0.5, color='r')
ax.set_xlabel('Time_in_Years',fontdict={'fontsize': 14, 'fontweight': 'bold'})
ax.set_ylabel('Stock Price',fontdict={'fontsize': 14, 'fontweight': 'bold'})
ax.set_title('Stock Path Simulations',fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.savefig(file_path, dpi=300, bbox_inches='tight')
plt.tight_layout()



# PLottin the stock price distribtuion ideally it shpould follow the lognormal distribution
stock=path_generations(10000,100,s0,strikes[0],risk_free_rate,volatility,Duration)['stock_T']
                        # fittin the stovk wioth lognormal fit 
shape, loc, scale = stats.lognorm.fit(stock, floc=0)
x=np.linspace(stock.min(), stock.max(), 1000)
pdf=stats.lognorm.pdf(x, shape, loc, scale)

plt.style.use('dark_background')
fig, ax = plt.subplots(figsize=(12, 8))
file_path=os.path.join(results_dir, 'stock_price_distribution.png')
ax.hist(stock, bins=50, density=True, alpha=0.6, color='g', edgecolor='black',label='Stock Price Distribution_from_Simulation')
ax.plot(x, pdf, 'r-', lw=2, label='Lognormal Fit')
ax.set_xlabel('Stock Price',fontdict={'fontsize': 14, 'fontweight': 'bold'})
ax.set_ylabel('Density',fontdict={'fontsize': 14, 'fontweight': 'bold'})
ax.set_title('Stock Price Distribution',fontdict={'fontsize': 14, 'fontweight': 'bold'})
ax.legend()
ax.grid(True, alpha=0.5, color='r')
plt.savefig(file_path, dpi=300, bbox_inches='tight')
plt.tight_layout()


In [ ]:
                                    # DAY 6 BINOMIAL OPTION PRICING FOR AMERICAN OPTIONS 
                                    # IMPORTING THE LIBRARIES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats


                                    # DEFINE THE PARAMTERS 
STOCK_INITIAL=100                   # INITILA STOCK
STRIKE_PRICES=np.linspace(80,200,20) # strike prices
STOCK_VOL=0.25                       # VOLATILITY
risk_free_rate=0.05                  # risk free rate 
Duration=1                           # time to maturity
TIME_STEPS=100                       # number of binomial time steps




                                    # DEFINING THE BNOMIAL GTREE GENRATOR
def binomial_tree(STOCK_INITIAL,STOCK_VOL,risk_free_rate,Duration,STEPS):
    dt=Duration/STEPS
    u=np.exp(STOCK_VOL*np.sqrt(dt))
    d=np.exp(-STOCK_VOL*np.sqrt(dt))
    p=(np.exp(risk_free_rate*dt)-d)/(u-d)
    PARAMTERS={}
    STOCKS=np.zeros((STEPS+1,STEPS+1))
    STOCKS[0,0]=STOCK_INITIAL
    for i in range(1,STEPS+1):
        for j in range(i+1):
            STOCKS[i,j]=STOCK_INITIAL*(u**j)*(d**(i-j))
    
    PARAMTERS['STOCK']=np.flip(STOCKS,axis=1)
    PARAMTERS['u']=u
    PARAMTERS['d']=d
    PARAMTERS['p']=p
    
    return PARAMTERS

# DEFINING THE AMERCIAN OPTION PRICING FUNCTION
def AOCC(STOCK_INITIAL,STRIKE_PRICES,STOCK_VOL,risk_free_rate,Duration,STEPS):
    PARAMETERS=binomial_tree(STOCK_INITIAL,STOCK_VOL,risk_free_rate,Duration,STEPS)
    STOCKS=PARAMETERS['STOCK']
    PRICING_MATRIX=np.zeros((STEPS+1,STEPS+1))
    PRICING_MATRIX[STEPS,:]=np.maximum(STOCKS[STEPS,:]-STRIKE_PRICES,0)
    p=PARAMETERS['p']
    dt=Duration/STEPS
    
    for i in range(STEPS-1,0,-1):
        for j in range(i+1):
            PRICING_MATRIX[i,j]=np.exp(-risk_free_rate*dt)*(p*PRICING_MATRIX[i+1,j+1]+(1-p)*PRICING_MATRIX[i+1,j-1])
            
    return PRICING_MATRIX

PRICING_MATRIX=AOCC(STOCK_INITIAL,100,STOCK_VOL,risk_free_rate,Duration,TIME_STEPS)
# SAVING THE AERCIAN OPTION PRICNING MATRIX IN CSV FILE
file_path=os.path.join(results_dir,'PRICING_MATRIX.csv')
columns=[f'{j}_value_of_option_for {j}_stock_path' for j in range(1,STEPS+2)]
index=[f'{i}_possible_time_step' for i in range(1,STEPS+2)]
df=pd.DataFrame(data=PRICING_MATRIX,index=index,columns=columns)
df.index.name='TIME_STEPS'
df.to_csv(file_path)




In [ ]:
                                                  # DAY 7 STOCHASTIC VOLALITLITY MODEL HESTON MODEL
# IMPORTING LIBRARIES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats


# Defining paramaters for Heston Model
S_0=100                               # Initial stock price
V_0=0.04                              # INITIAL VARIENCE
r=0.03                                # Risk Free Rate
K=2                                   # speed pof mean reversion
THETA=0.04                            # long term varience
SIGMA=0.2                             # volatility of volatilty
RHO=-0.7                              # correlation coefficient
T=1                                   # Time to maturity
STRIKES=np.linspace(50,200,30)        # STRIKE PRICES
PRICES=pd.DataFrame(index=STRIKES)    # DATAFRAME FOR STRIKE PRICES
NOOFTIMESTEPS=100                     # NUMBER OF TIMESTEPS
NOOFPATHS=10000                 # NUMBER OF PATHS


def path_generator_heston(S_0,V_0,r,K,THETA,SIGMA,RHO,T,Noofpaths,Nooftimesteps):
    dt=T/Nooftimesteps
    S=np.zeros((Noofpaths,Nooftimesteps+1))
    V=np.zeros((Noofpaths,Nooftimesteps+1))
    S[:,0]=S_0
    V[:,0]=V_0
    z1=np.random.randn(Noofpaths,Nooftimesteps)
    z2=np.random.randn(Noofpaths,Nooftimesteps)
    dW_s=np.sqrt(dt)*z1
    dW_v=np.sqrt(dt)*(z1*RHO + np.sqrt(1-RHO**2)*z2)
    W_s=np.cumsum(dW_s,axis=1)
    W_v=np.cumsum(dW_v,axis=1)
    TIME=[i*dt for i in range(Nooftimesteps+1)]

    PATHS={}
    for t in range(1,Nooftimesteps+1):
        
        V[:,t]=V[:,t-1]+K*(THETA-V[:,t-1])*dt+SIGMA*np.sqrt(V[:,t-1])*dW_v[:,t-1]
        V[:,t]=np.maximum(V[:,t],0)
        S[:,t]=S[:,t-1] + S[:,t-1]*r*dt + np.power(V[:,t-1],0.5)*S[:,t-1]*dW_s[:,t-1]
        S[:,t]=np.maximum(S[:,t],0)
       
    PATHS['STOCKS']=S
    PATHS['STOCK_MATURITY']=S[:,-1]
    PATHS['VARIENCE']=V
    PATHS['VAR_MATURITY']=V[:,-1]
    PATHS['TIME']=TIME
    return PATHS


def monte_carlo_pricing(type,STRIKE,S_0,V_0,r,K,THETA,SIGMA,RHO,T,Noofpaths,Nooftimesteps):
    paramters=path_generator_heston(S_0,V_0,r,K,THETA,SIGMA,RHO,T,Noofpaths,Nooftimesteps)
    stocks=paramters['STOCKS']
    volatilities=paramters['VARIENCE']
    type=type.lower()
    if type=='call':
        price=np.mean(np.maximum(0,STRIKE-stocks))*np.exp(-r*T)
    elif type=='put':
        price=np.mean(np.maximum(0,stocks-STRIKE))*np.exp(-r*T)
    return price
  
PRICES['CALL_MONTE_CARLO']=[monte_carlo_pricing('call',STRIKE,S_0,V_0,r,K,THETA,SIGMA,RHO,T,NOOFPATHS,NOOFTIMESTEPS) for STRIKE in STRIKES]
PRICES['PUT_MONTE_CARLO']=[monte_carlo_pricing('put',STRIKE,S_0,V_0,r,K,THETA,SIGMA,RHO,T,NOOFPATHS,NOOFTIMESTEPS) for STRIKE in STRIKES]
PRICES.index.name='STRIKE PRICES'
file_path=os.path.join(results_dir, 'monte_carlo_prices_Heston_model.csv')
PRICES.to_csv(file_path)


                            # PLOTTING THE STOCK PATHS THE VOLAILTITY PATHS AND THE CALL AND PUT PRICES

                                                    # PLOTTING THE SIMULATED STOCK PATHS

plt.style.use('dark_background')
fig, ax = plt.subplots(figsize=(12, 8))
file_path=os.path.join(results_dir, 'stock_path_simulations_Heston_model.png')
paths=path_generator_heston(S_0,V_0,r,K,THETA,SIGMA,RHO,T,NOOFPATHS,NOOFTIMESTEPS)['STOCKS']
time=path_generator_heston(S_0,V_0,r,K,THETA,SIGMA,RHO,T,NOOFPATHS,NOOFTIMESTEPS)['TIME']
for i in range(100):
    ax.plot(time, paths[i,:], color='royalblue', alpha=0.8)
ax.grid(True, alpha=0.5, color='r')
ax.set_xlabel('Time_in_Years',fontdict={'fontsize': 14, 'fontweight': 'bold'})
ax.set_ylabel('Stock Price',fontdict={'fontsize': 14, 'fontweight': 'bold'})
ax.set_title('Stock Path Simulations',fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.tight_layout()
plt.savefig(file_path, dpi=300, bbox_inches='tight')





                                                        # PLOTTING THE VOLATILITY PATHS

plt.style.use('dark_background')
fig, ax = plt.subplots(figsize=(12, 8))
file_path=os.path.join(results_dir, 'stochastic_variance_path_simulations_Heston_model.png')
paths=path_generator_heston(S_0,V_0,r,K,THETA,SIGMA,RHO,T,NOOFPATHS,NOOFTIMESTEPS)['VARIENCE']
time=path_generator_heston(S_0,V_0,r,K,THETA,SIGMA,RHO,T,NOOFPATHS,NOOFTIMESTEPS)['TIME']
for i in range(100):
    ax.plot(time, paths[i,:], color='green', alpha=0.8)
ax.grid(True, alpha=0.5, color='r')
ax.set_xlabel('Time_in_Years',fontdict={'fontsize': 14, 'fontweight': 'bold'})
ax.set_ylabel('VARIENCE',fontdict={'fontsize': 14, 'fontweight': 'bold'})
ax.set_title('StOCHASTIC VARIENCE SIMULATIONS',fontdict={'fontsize': 14, 'fontweight': 'bold'})
plt.tight_layout()
plt.savefig(file_path, dpi=300, bbox_inches='tight')




                                                       # PLOTTING THE CALL AND PUT PRICES FOR DIFFERENT STRIKES


plt.style.use('dark_background')
fig, ax = plt.subplots(figsize=(12, 8))
fig_path=os.path.join(results_dir, 'call_and_put_prices_Heston_model.png')
ax.plot(STRIKES, PRICES['CALL_MONTE_CARLO'], color='royalblue', alpha=0.8,label='CALL_PRICE')
ax.plot(STRIKES, PRICES['PUT_MONTE_CARLO'], color='green', alpha=0.8,label='PUT_PRICE')
ax.grid(True, alpha=0.5, color='r')
ax.set_xlabel('STRIKE PRICES',fontdict={'fontsize': 14, 'fontweight': 'bold'})
ax.set_ylabel('PRICE',fontdict={'fontsize': 14, 'fontweight': 'bold'})
ax.set_title('CALL AND PUT PRICES_Heston Model')
plt.tight_layout()
plt.legend(loc='upper left',frameon=True,fontsize=12,facecolor='BLUE')
plt.savefig(fig_path, dpi=300, bbox_inches='tight')

